In [6]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [7]:
adeft_grounder = AdeftGrounder()

In [8]:
shortforms = ['AD']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [9]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [11]:
list(zip(longforms, counts))

[('alzheimer s disease', 15075),
 ('atopic dermatitis', 1814),
 ('alcohol dependence', 191),
 ('axial diffusivity', 157),
 ('activation domain', 151),
 ('anaerobic digestion', 133),
 ('autosomal dominant', 114),
 ('alzheimer s dementia', 69),
 ('aortic dissection', 64),
 ('antidepressant', 62),
 ('androstenedione', 55),
 ('androgen deprivation', 51),
 ('autonomic dysreflexia', 50),
 ('afterdischarge', 50),
 ('adenocarcinoma', 42),
 ('adenosine', 38),
 ('actinomycin d', 35),
 ('anoxic depolarization', 30),
 ('aortic distensibility', 27),
 ('androgen dependence', 27),
 ('adiponectin', 22),
 ('atherogenic diet', 21),
 ('adrenaline', 21),
 ('amiodarone', 19),
 ('adriamycin', 17),
 ('antidepressant drugs', 17),
 ('anxiety disorders', 17),
 ('autistic disorders', 17),
 ('atherogenic dyslipidemia', 13),
 ('acid domain', 13),
 ('disease such as alzheimer s', 13),
 ('average density', 11),
 ('after discharge', 11),
 ('acetabular dysplasia', 10),
 ('alendronate', 10),
 ('disease including alzhei

In [12]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [13]:
names

{'MESH:D003876': 'Dermatitis, Atopic',
 'MESH:D000437': 'Alcoholism',
 'GO:GO:0007586': 'digestion',
 'HP:HP:0000006': 'Autosomal dominant inheritance',
 'MESH:D000544': 'Alzheimer Disease',
 'HP:HP:0002647': 'Aortic dissection',
 'CHEBI:CHEBI:35469': 'antidepressant',
 'CHEBI:CHEBI:16422': 'androst-4-ene-3,17-dione',
 'MESH:D020211': 'Autonomic Dysreflexia',
 'MESH:D000230': 'Adenocarcinoma',
 'CHEBI:CHEBI:16335': 'adenosine',
 'CHEBI:CHEBI:27666': 'actinomycin D',
 'HGNC:13633': 'ADIPOQ',
 'MESH:D004036': 'Diet, Atherogenic',
 'CHEBI:CHEBI:33568': 'adrenaline',
 'CHEBI:CHEBI:2663': 'amiodarone',
 'CHEBI:CHEBI:28748': 'doxorubicin',
 'MESH:D001008': 'Anxiety Disorders',
 'MESH:D050171': 'Dyslipidemias',
 'CHEBI:CHEBI:2567': 'alendronic acid',
 'MESH:D004391': 'Dust',
 'MESH:D010361': 'Patients',
 'MESH:D004238': 'Diverticulitis',
 'CHEBI:CHEBI:40519': 'adamantane',
 'MESH:D003704': 'Dementia',
 'MESH:D055864': 'Droughts',
 'MESH:D017667': 'Adipocytes',
 'MESH:D000236': 'Adenoma',
 'CH

In [14]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-11 17:04:48] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [15]:
result = [grounding_map, names, pos_labels]

In [16]:
result

[{'abdominal diameter': 'ungrounded',
  'academic detail': 'ungrounded',
  'access deficits': 'ungrounded',
  'acetabular dysplasia': 'ungrounded',
  'achieved drive': 'ungrounded',
  'acid': 'ungrounded',
  'acid domain': 'ungrounded',
  'aclidinium bromide': 'CHEBI:CHEBI:65344',
  'acromicric dysplasia': 'MESH:C535662',
  'actinomycin d': 'CHEBI:CHEBI:27666',
  'activation diet': 'ungrounded',
  'activation domain': 'ungrounded',
  'acute diverticulitis': 'MESH:D004238',
  'ad': 'ungrounded',
  'ad libitum': 'ungrounded',
  'ad patients': 'ungrounded',
  'adamantane': 'CHEBI:CHEBI:40519',
  'adaptive dose': 'ungrounded',
  'adefovir dipivoxil': 'MESH:C106812',
  'adenocarcinoma': 'ungrounded',
  'adenoma': 'MESH:D000236',
  'adenosine': 'ungrounded',
  'adenosine deaminase': 'ungrounded',
  'adenovirus': 'MESH:D000256',
  'adequate dairy': 'ungrounded',
  'adherent': 'ungrounded',
  'adipocyte': 'MESH:D017667',
  'adipogenesis': 'MESH:D050156',
  'adipogenic': 'MESH:D050156',
  'adip

In [18]:
grounding_map, names, pos_labels = [{'abdominal diameter': 'ungrounded',
  'academic detail': 'ungrounded',
  'access deficits': 'ungrounded',
  'acetabular dysplasia': 'ungrounded',
  'achieved drive': 'ungrounded',
  'acid': 'ungrounded',
  'acid domain': 'ungrounded',
  'aclidinium bromide': 'CHEBI:CHEBI:65344',
  'acromicric dysplasia': 'MESH:C535662',
  'actinomycin d': 'CHEBI:CHEBI:27666',
  'activation diet': 'ungrounded',
  'activation domain': 'ungrounded',
  'acute diverticulitis': 'MESH:D004238',
  'ad': 'ungrounded',
  'ad libitum': 'ungrounded',
  'ad patients': 'ungrounded',
  'adamantane': 'CHEBI:CHEBI:40519',
  'adaptive dose': 'ungrounded',
  'adefovir dipivoxil': 'MESH:C106812',
  'adenocarcinoma': 'ungrounded',
  'adenoma': 'MESH:D000236',
  'adenosine': 'ungrounded',
  'adenosine deaminase': 'ungrounded',
  'adenovirus': 'MESH:D000256',
  'adequate dairy': 'ungrounded',
  'adherent': 'ungrounded',
  'adipocyte': 'MESH:D017667',
  'adipogenesis': 'MESH:D050156',
  'adipogenic': 'MESH:D050156',
  'adiponectin': 'HGNC:13633',
  'adipose': 'ungrounded',
  'adipose tissue': 'ungrounded',
  'adrenalectomy': 'MESH:D000315',
  'adrenaline': 'MESH:D004837',
  'adriamycin': 'CHEBI:CHEBI:28748',
  'adult': 'ungrounded',
  'advanced directly': 'ungrounded',
  'adventitial diameter': 'ungrounded',
  'aerosol deposition': 'ungrounded',
  'affected disorders': 'ungrounded',
  'after discharge': 'ungrounded',
  'afterdischarge': 'ungrounded',
  'afterdischarge duration': 'ungrounded',
  'agelasine d': 'MESH:C509934',
  'alcohol dehydrogenase': 'ungrounded',
  'alcohol dependence': 'MESH:D000437',
  'aldehyde decarbonylase': 'ungrounded',
  'alendronate': 'CHEBI:CHEBI:2567',
  'alexidine dihydrochloride': 'ungrounded',
  'allergic dermatitis': 'MESH:D003876',
  'alzheimer dementia': 'MESH:D000544',
  'alzheimer disease s': 'MESH:D000544',
  'alzheimer s dementia': 'MESH:D000544',
  'alzheimer s dis ease': 'MESH:D000544',
  'alzheimer s disease': 'MESH:D000544',
  'alzheimer s disease dementia': 'MESH:D000544',
  'alzheimer s disease group': 'MESH:D000544',
  'alzheimer s disease patients': 'MESH:D000544',
  'alzheimer type dementia': 'MESH:D000544',
  'ambulance diverse': 'ungrounded',
  'amiodarone': 'CHEBI:CHEBI:2663',
  'amorpha 4 11 diene': 'ungrounded',
  'amphetamine dependence': 'ungrounded',
  'amplified drought': 'ungrounded',
  'anaerobic digestion': 'ungrounded',
  'anchorage dependence': 'ungrounded',
  'anchoring domain': 'ungrounded',
  'androgen deficiency': 'ungrounded',
  'androgen dependence': 'ungrounded',
  'androgen depletion': 'ungrounded',
  'androgen deprivation': 'ungrounded',
  'andrographolide': 'CHEBI:CHEBI:65408',
  'androstene 3 17 dione': 'ungrounded',
  'androstenedione': 'MESH:D000735',
  'androsterone': 'CHEBI:CHEBI:16032',
  'anesthetic depth': 'ungrounded',
  'anoxic depolarization': 'ungrounded',
  'antenatal depression': 'ungrounded',
  'anterior deltoid': 'ungrounded',
  'anti desensitizer': 'ungrounded',
  'antidepressant': 'CHEBI:CHEBI:35469',
  'antidepressant drugs': 'CHEBI:CHEBI:35469',
  'anxiety depression': 'ungrounded',
  'anxiety disorders': 'MESH:D001008',
  'anxious depression': 'ungrounded',
  'aortic diameter': 'ungrounded',
  'aortic dilation': 'ungrounded',
  'aortic dissection': 'ungrounded',
  'aortic distensibility': 'ungrounded',
  'aortic distensibilty': 'ungrounded',
  'apical domain': 'ungrounded',
  'approximately digestion': 'ungrounded',
  'aqueous depth': 'ungrounded',
  'area based breast density': 'MESH:D000071060',
  'arginine deiminase': 'ungrounded',
  'arrhythmic death': 'ungrounded',
  'articular disc': 'ungrounded',
  'artificial dermis': 'ungrounded',
  'asian dust': 'ungrounded',
  'assembly domain': 'ungrounded',
  'asymmetric dihydroxylation': 'ungrounded',
  'at diagnosis': 'ungrounded',
  'atherogenic diet': 'ungrounded',
  'atherogenic dyslipidaemia': 'ungrounded',
  'atherogenic dyslipidemia': 'ungrounded',
  'atopic dermatitis': 'MESH:D003876',
  'atopic like dermatitis': 'MESH:D003876',
  'autistic disorders': 'ungrounded',
  'autogenic drainage': 'ungrounded',
  'autoimmune disorders': 'ungrounded',
  'autonomic dysfunction': 'MESH:D054969',
  'autonomic dysreflexia': 'ungrounded',
  'autosomal dominant': 'ungrounded',
  'average density': 'ungrounded',
  'axial diffusivity': 'ungrounded',
  'axillary dissection': 'ungrounded',
  'alzheimer s': 'MESH:D000544',
  'md axial': 'ungrounded'},
 {'CHEBI:CHEBI:65344': 'aclidinium bromide',
  'MESH:C535662': 'Acromicric dysplasia',
  'CHEBI:CHEBI:27666': 'actinomycin D',
  'MESH:D004238': 'Diverticulitis',
  'CHEBI:CHEBI:40519': 'adamantane',
  'MESH:C106812': 'adefovir dipivoxil',
  'MESH:D000236': 'Adenoma',
  'MESH:D000256': 'Adenoviridae',
  'MESH:D017667': 'Adipocytes',
  'MESH:D050156': 'Adipogenesis',
  'HGNC:13633': 'ADIPOQ',
  'MESH:D000315': 'Adrenalectomy',
  'MESH:D004837': 'Epinephrine',
  'CHEBI:CHEBI:28748': 'doxorubicin',
  'MESH:C509934': 'agelasine D',
  'MESH:D000437': 'Alcoholism',
  'CHEBI:CHEBI:2567': 'alendronic acid',
  'MESH:D003876': 'Dermatitis, Atopic',
  'MESH:D000544': 'Alzheimer Disease',
  'CHEBI:CHEBI:2663': 'amiodarone',
  'CHEBI:CHEBI:65408': 'andrographolide',
  'MESH:D000735': 'Androstenedione',
  'CHEBI:CHEBI:16032': 'androsterone',
  'CHEBI:CHEBI:35469': 'antidepressant',
  'MESH:D001008': 'Anxiety Disorders',
  'MESH:D000071060': 'Breast Density',
  'MESH:D054969': 'Primary Dysautonomias'},
 ['HGNC:13633', 'MESH:D000544']]

In [19]:
excluded_longforms = ['ad', 'ad patients']

In [20]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [21]:
additional_entities = {}

In [22]:
unambiguous_agent_texts = {}

In [23]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [24]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [25]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [26]:
intersection1

[]

In [27]:
intersection2

[]

In [28]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [29]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [30]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-11 17:24:41] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-11 17:34:35] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9888577488339235 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [31]:
classifier.stats

{'label_distribution': {'ungrounded': 888,
  'MESH:D004837': 18,
  'CHEBI:CHEBI:27666': 27,
  'MESH:D003876': 1293,
  'MESH:D000544': 9968,
  'CHEBI:CHEBI:35469': 61,
  'MESH:D000735': 44,
  'CHEBI:CHEBI:2663': 12,
  'CHEBI:CHEBI:28748': 10,
  'MESH:D000315': 3,
  'CHEBI:CHEBI:16032': 2,
  'MESH:D000256': 5,
  'MESH:D000437': 122,
  'MESH:D017667': 4,
  'MESH:D001008': 14,
  'HGNC:13633': 11,
  'MESH:D050156': 3,
  'MESH:C106812': 4,
  'MESH:D000236': 5,
  'CHEBI:CHEBI:40519': 4,
  'MESH:C535662': 5,
  'CHEBI:CHEBI:65408': 5,
  'MESH:D054969': 4,
  'CHEBI:CHEBI:2567': 2,
  'MESH:C509934': 1,
  'MESH:D004238': 6,
  'MESH:D000071060': 1,
  'CHEBI:CHEBI:65344': 1},
 'f1': {'mean': 0.988858, 'std': 0.00071},
 'precision': {'mean': 0.982968, 'std': 0.000739},
 'recall': {'mean': 0.994889, 'std': 0.000802},
 'MESH:D000544': {'f1': {'mean': 0.989481, 'std': 0.000899},
  'pr': {'mean': 0.995586, 'std': 0.001071},
  'rc': {'mean': 0.983451, 'std': 0.000863}},
 'MESH:D000315': {'f1': {'mean': 0.

In [32]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [33]:
disamb.dump(model_name, results_path)

In [34]:
print(disamb.info())

Disambiguation model for AD

Produces the disambiguations:
	ADIPOQ*	HGNC:13633
	Acromicric dysplasia	MESH:C535662
	Adenoma	MESH:D000236
	Adenoviridae	MESH:D000256
	Adipocytes	MESH:D017667
	Adipogenesis	MESH:D050156
	Adrenalectomy	MESH:D000315
	Alcoholism	MESH:D000437
	Alzheimer Disease*	MESH:D000544
	Androstenedione	MESH:D000735
	Anxiety Disorders	MESH:D001008
	Breast Density	MESH:D000071060
	Dermatitis, Atopic	MESH:D003876
	Diverticulitis	MESH:D004238
	Epinephrine	MESH:D004837
	Primary Dysautonomias	MESH:D054969
	aclidinium bromide	CHEBI:CHEBI:65344
	actinomycin D	CHEBI:CHEBI:27666
	adamantane	CHEBI:CHEBI:40519
	adefovir dipivoxil	MESH:C106812
	agelasine D	MESH:C509934
	alendronic acid	CHEBI:CHEBI:2567
	amiodarone	CHEBI:CHEBI:2663
	andrographolide	CHEBI:CHEBI:65408
	androsterone	CHEBI:CHEBI:16032
	antidepressant	CHEBI:CHEBI:35469
	doxorubicin	CHEBI:CHEBI:28748

Class level metrics:
--------------------
Grounding            	Count	F1     
    Alzheimer Disease*	9968	0.98948
   Dermatit

In [35]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1